<a href="https://colab.research.google.com/github/singwang-cn/Neural-Network/blob/master/Implementation_of_LSTM_with_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

In [15]:
rnn = torch.nn.LSTM(10, 20, 4, dropout=0.2)
input = torch.randn(5, 3, 10)
h0 = torch.randn(4, 3, 20)
c0 = torch.randn(4, 3, 20)
output, (hn, cn) = rnn(input, (h0, c0))

In [16]:
cn

tensor([[[-0.0758,  0.1166, -0.3063, -0.4799,  0.2947,  0.1360, -0.0791,
           0.1712, -0.2164, -0.3865, -0.1178, -0.1708,  0.0048, -0.2736,
           0.2347,  0.0883,  0.0991,  0.1566, -0.2101, -0.2663],
         [ 0.1959,  0.2446, -0.0243, -0.6108,  0.2021,  0.0291,  0.0601,
           0.1985, -0.2383, -0.3295, -0.1489, -0.0097,  0.0025, -0.2148,
           0.0711,  0.0853,  0.4079,  0.2149, -0.2774, -0.4469],
         [-0.1337,  0.4927,  0.3490, -0.2830,  0.2021, -0.0405,  0.1181,
           0.0858, -0.0068, -0.2568, -0.3134, -0.1022, -0.0970, -0.3719,
          -0.2710, -0.3467, -0.1562, -0.2175,  0.0763, -0.2983]],

        [[-0.1441, -0.0026, -0.0551, -0.0094,  0.1027,  0.5276, -0.0341,
           0.3468, -0.0455,  0.1430,  0.1451,  0.3400, -0.1778,  0.1638,
          -0.3072,  0.0820, -0.0297, -0.2953, -0.1843,  0.2146],
         [-0.1332, -0.0015, -0.1021, -0.0969,  0.2013,  0.3413,  0.1021,
           0.1919, -0.1895,  0.0096,  0.1304,  0.3971, -0.2486,  0.2551,
        